In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv('CSV/TransfixContacts.csv',  dtype=str)
print(df.shape)
df.head(2)

(76696, 2)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Id,Email
0,0031J00001UFWmbQAH,bdstreelman@aol.com
1,0031J00001XMYEKQA5,nancy@kiwicrate.com


In [2]:
# Clean the data
df.dropna()
# df['Email'] = df['Email'].str.replace('[^a-zA-Z]', '')
# df['Email'] = df['Email'].str.replace(r'[^\w\s]+', '')

contact_emails = df['Email']

In [3]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


In [4]:
# get Tf-IDF Matrix
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(contact_emails.apply(lambda x: np.str_(x)))


In [5]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [6]:
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7)


In [7]:
def get_matches_df(sparse_matrix, email_vector,email_ids, top=5):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    left_name_Ids = np.empty([nr_matches], dtype=object)
    right_name_Ids = np.empty([nr_matches], dtype=object)
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(nr_matches):        
        left_name_Ids[index] = email_ids[sparserows[index]]
        left_side[index] = email_vector[sparserows[index]]

        right_name_Ids[index] = email_ids[sparsecols[index]]
        right_side[index] = email_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({
                        'SFDC_ID':  left_name_Ids,
                        'left_side': left_side,
                        'right_SFDC_ID':right_name_Ids,
                          'right_side': right_side,
                           'similairity': similairity})

In [8]:
name_Ids = df['Id']
matches_df = get_matches_df(matches, contact_emails,name_Ids, top=76696)






In [9]:
matches_df = matches_df[matches_df['similairity'] > 0.9] 




In [10]:
matches_df.sort_values('SFDC_ID' )



,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
2231,0031J00001B0zPpQAJ,rculver@condadogroup.com,0031J00001B0zPpQAJ,rculver@condadogroup.com,1.0
2236,0031J00001B1GzJQAV,cs@pegasushomefashions.com,0031J00001B1GzJQAV,cs@pegasushomefashions.com,1.0
2235,0031J00001B1LpLQAV,kates@vivbev.com,0031J00001B1LpLQAV,kates@vivbev.com,1.0
2239,0031J00001B1QMEQA3,lisa.temple@tileshop.com,0031J00001B1QMEQA3,lisa.temple@tileshop.com,1.0
2287,0031J00001B35YEQAZ,dan.drieselman@searshc.com,0031J00001B35YEQAZ,dan.drieselman@searshc.com,1.0
...,...,...,...,...,...
248,003o000001Dzy0QAAR,steve.haver@huffy.com,003o000001Dzy0QAAR,steve.haver@huffy.com,1.0
246,003o000001E00PxAAJ,esolorzano@entecresins.com,003o000001E00PxAAJ,esolorzano@entecresins.com,1.0
756,003o000001E0UclAAF,paul.farber@truevalue.com,003o000001E0UclAAF,paul.farber@truevalue.com,1.0
757,003o000001E0UcmAAF,christopher.galletto@truevalue.com,003o000001E0UcmAAF,christopher.galletto@truevalue.com,1.0


In [11]:
mask = matches_df['SFDC_ID'] == matches_df['right_SFDC_ID'] 

#if mask is true item in df['SFDC_ID'] is selected else item in df['right_SFDC_ID'] is selected

matches_df[mask]



,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
0,0031J00001UFWmbQAH,bdstreelman@aol.com,0031J00001UFWmbQAH,bdstreelman@aol.com,1.0
1,0031J00001XMYEKQA5,nancy@kiwicrate.com,0031J00001XMYEKQA5,nancy@kiwicrate.com,1.0
2,0031J00001P6JZ3QAN,jjakobsen@singlesource.com,0031J00001P6JZ3QAN,jjakobsen@singlesource.com,1.0
3,0031J00001L1KoJQAV,takashi.tobiya@sumitomocorp.com,0031J00001L1KoJQAV,takashi.tobiya@sumitomocorp.com,1.0
4,0031J00001XMYYKQA5,teresa.niekerk@allisontransmission.com,0031J00001XMYYKQA5,teresa.niekerk@allisontransmission.com,1.0
...,...,...,...,...,...
76687,0031J00001kMObcQAG,ipsnagra@yahoo.com,0031J00001kMObcQAG,ipsnagra@yahoo.com,1.0
76689,0031J00001kMObdQAG,rodney.carter0351@gmail.com,0031J00001kMObdQAG,rodney.carter0351@gmail.com,1.0
76691,0031J00001kMObeQAG,hytrucking@gmail.com,0031J00001kMObeQAG,hytrucking@gmail.com,1.0
76693,0031J00001kMObkQAG,ohernandez@rjlogistics.com,0031J00001kMObkQAG,ohernandez@rjlogistics.com,1.0


In [12]:
# Get names of indexes for which column mask == true
indexNames = matches_df[ mask ].index
# Delete these row indexes from dataFrame
matches_df.drop(indexNames , inplace=True)



In [13]:
matches_df.sort_values('similairity',ascending=True)



,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
61475,0031J00001epViSQAU,p.metz@vilebrequin.com,0031J00001epVihQAE,f.metz@vilebrequin.com,0.901055
61728,0031J00001epVihQAE,f.metz@vilebrequin.com,0031J00001epViSQAU,p.metz@vilebrequin.com,0.901055
16063,0031J00001O5HGyQAN,jmiller@buchananhauling.com,0031J00001O5HFUQA3,cmiller@buchananhauling.com,0.901449
18452,0031J00001O5HFUQA3,cmiller@buchananhauling.com,0031J00001O5HGyQAN,jmiller@buchananhauling.com,0.901449
30838,0031J00001QEnL2QAL,tcipiti@vallartasupermarkets.com,0031J00001kMM82QAG,tim.cipiti@vallartasupermarkets.com,0.901950
...,...,...,...,...,...
48451,0031J00001emghIQAQ,safety@sweetland.com,0031J00001kMPpxQAG,safety@sweetland.com,1.000000
44873,0031J00001eoXxUQAU,tirby@hwyboundtrucking.com,0031J00001kMQLVQA4,tirby@hwyboundtrucking.com,1.000000
51665,0031J00001em8tIQAQ,dispatchpiperlogistics@gmail.com,0031J00001kMPJTQA4,dispatchpiperlogistics@gmail.com,1.000000
75770,0031J00001kMQLVQA4,tirby@hwyboundtrucking.com,0031J00001eoXxUQAU,tirby@hwyboundtrucking.com,1.000000


In [14]:
matches_df.to_csv ('./Results/2020-03-10_ContactDups.csv',index = False, header=True)

In [15]:
mask = matches_df['SFDC_ID'] < matches_df['right_SFDC_ID'] 

# creates a new column checking True vs False, 

#if mask is true item in df['SFDC_ID'] is selected else item in df['right_SFDC_ID'] is selected

matches_df['col1'] = matches_df['SFDC_ID'].where(mask, matches_df['right_SFDC_ID'])

#same as above but a column for df['right_SFDC_ID']
matches_df['col2'] = matches_df['right_SFDC_ID'].where(mask, matches_df['SFDC_ID'])

# checks for duplicates in `col1` and `col2` and removes last duplicate
matches_df = matches_df.drop_duplicates(subset=['col1'])



In [16]:
matches_df.sort_values('similairity',ascending=True)

matches_df['similairity']= matches_df['similairity'].astype(str)


In [17]:
matches_df.to_csv ('./Results/2020-03-10_ContactDups2.csv',index = False, header=True)



In [18]:
test_df = matches_df.groupby(['SFDC_ID', 'left_side' ], as_index=False)[['right_SFDC_ID', 'right_side', 'similairity']].agg(lambda x: ','.join(x))

test_df.head(500)


,SFDC_ID,left_side,right_SFDC_ID,right_side,similairity
0,0031J00001O5HGyQAN,jmiller@buchananhauling.com,"0031J00001O5Hv0QAF,0031J00001O5HFUQA3","dmiller@buchananhauling.com,cmiller@buchananhauling.com","0.9144917996221732,0.9014494273103513"
1,0031J00001OimvwQAB,mikerobinson@carlislefsp.com,003o000000wObweAAC,mikerobinson@carlislefsp.com,1.0
2,0031J00001Orn8kQAB,lschack@thecheesecakefactory.com,0031J00001enwXRQAY,risack@thecheesecakefactory.com,0.9020537779510003
3,0031J00001P48IHQAZ,dkaduke@limitedbrands.com,0031J00001P48MnQAJ,dkaduke@limitedbrands.com,1.0000000000000002
4,0031J00001P48McQAJ,robert.mcrae@pepsico.com,0031J00001T50Z3QAJ,robert.mcrae@pepsico.com,1.0
...,...,...,...,...,...
495,0031J00001cv4apQAA,greg@logisticsetc.com,0031J00001kMP2AQAW,greg@logisticsetc.com,1.0000000000000002
496,0031J00001cv4aqQAA,cculler@redwoodlogistics.com,0031J00001kMP2BQAW,cculler@redwoodlogistics.com,1.0000000000000004
497,0031J00001cv4arQAA,jhansen@wisnat.com,0031J00001kMP3pQAG,jhansen@wisnat.com,1.0000000000000004
498,0031J00001cv4asQAA,mfounguic@gmail.com,0031J00001kMOztQAG,mfounguic@gmail.com,1.0000000000000002


In [19]:
test_df.to_csv ('./Results/2020-03-10_ContactDups3.csv',index = False, header=True)

